In [ ]:
def criar_database():
  import sqlite3

  try:
    conn = sqlite3.connect('alunos.db')
    cursor = conn.cursor()
    print('Database Created')

    sql = 'select sqlite_version()'
    cursor.execute(sql)
    results = cursor.fetchall()
    print('Version: ', results)
    cursor.close()

  except sqlite3.Error as e:
    print('Connection Error, or cursor problem', e)

  finally:
    if(conn):
      conn.close()
      print('Connection Closed')

def criar_tabela():
  import sqlite3

  try:
    conn = sqlite3.connect('alunos.db')
    
    sql = '''CREATE TABLE alunos (
          DRE INTEGER NOT NULL,
          Curso TEXT NOT NULL,
          Nome TEXT NOT NULL,
          Genero TEXT NOT NULL,
          Data_de_Nascimento BLOB NOT NULL,
          Altura REAL NOT NULL,
          Peso REAL NOT NULL,
          CRA REAL NOT NULL,
          Creditos_Obtidos INTEGER NOT NULL,
          Renda REAL NOT NULL);'''

    cursor = conn.cursor()
    cursor.execute(sql)
    conn.commit()
    print('Table Created')
    cursor.close()

  except sqlite3.Error as e:
      print('Connection Error, or cursor problem', e)

  finally:
      if(conn):
          conn.close()
          print('Connection Closed')

def inserir_dados():
  import sqlite3

  print('SQLite Custom Inserter v0.1')
  print('-'*79)

  while True:
      conn = sqlite3.connect('alunos.db')
      
      try:
          check = input("Enter 'quit' to Break: ")

          if check == 'quit':
              print('Script Terminated')
              break
          
          dre = int(input('DRE: '))
          curso = input('Curso: ')
          nome = input('Nome: ')
          genero = input('Genero: ')
          data = input('Data de Nascimento: ')
          altura = float(input('Altura: '))
          peso = float(input('Peso: '))
          cra = float(input('CRA: '))
          creditos = int(input('Creditos Obtidos: '))
          renda = float(input('Renda: '))

          sql = '''INSERT INTO alunos
              (DRE, Curso, Nome, Genero, Data_de_Nascimento, Altura, Peso, CRA, Creditos_Obtidos, Renda)
              VALUES ('{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}');'''.format(
              dre, curso, nome, genero, data, altura, peso, cra, creditos, renda)

          cursor = conn.cursor()
          cursor.execute(sql)
          conn.commit()
          print('Insert Operation OK')
          cursor.close()

      except sqlite3.Error as e:
          print('Error while Inserting Data', e)

      finally:
          if(conn):
              conn.close()
              print('Connection Closed')

def dados_estatisticos():
  import sqlite3
  import pandas as pd
  import plotly.express as px

  conn = sqlite3.connect('alunos.db')
  qry = '''SELECT * FROM alunos'''
  df = pd.read_sql_query(qry, conn)
  dados_estatisticos = df.describe().iloc[:,1:]

  genero_qtd = df.groupby('Genero').count().iloc[:,0]
  fig1 = px.bar(genero_qtd,
                x=genero_qtd.index,
                y=genero_qtd.values,
                title='Quantidade de Alunos por Genero',
                color=genero_qtd.index,
                labels={'x':'Genero','y':'Quantidade'},
                text=genero_qtd.values,
                width=600, height=400)
  
  curso_qtd = df.groupby('Curso').count().iloc[:,0]
  fig2 = px.pie(curso_qtd,
                values=curso_qtd.values,
                names=curso_qtd.index,
                title='Quantidade de Alunos por Curso',
                width=600, height=400)
  fig2.update_traces(textinfo='value')
  
  cra = df.iloc[:,7]
  fig3 = px.histogram(cra, x=cra.values, range_x=[0,10],
                      title='Distribuicao do CRA dos Alunos',
                      labels={'x':'CRA'},
                      width=600, height=400)
  fig3.update_layout(yaxis_title='Quantidade')

  curso_media_cra = df.groupby('Curso').mean().iloc[:,3]
  fig4 = px.bar(curso_media_cra,
                      x=curso_media_cra.index,
                      y=curso_media_cra.values,
                      title='Media do CRA por Curso',
                      labels={'x':'Cursos','y':'CRA'},
                      text=curso_media_cra.values,
                      width=600, height=400)

  print(dados_estatisticos)
  print(fig1.show(), fig2.show(), fig3.show(), fig4.show())

criar_database()
criar_tabela()
inserir_dados()
dados_estatisticos()